Untitled.ipynb        sentiment-data-v2.csv
fastText              sentiment.ipynb


In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_csv('./data/sentiment-data-v2.csv')
data.shape

(3685, 2)

In [5]:
data.head()

,label,text
0,Negative,Lebih-lebih lagi dengan kemudahan internet da...
1,Positive,boleh memberi teguran kepada parti tetapi perl...
2,Negative,Adalah membingungkan mengapa masyarakat Cina b...
3,Positive,Kami menurunkan defisit daripada 6.7 peratus p...
4,Negative,"Ini masalahnya. Bukan rakyat, tetapi sistem"


In [6]:
train, valid = train_test_split(data, test_size=0.2, random_state=42)

# train.to_csv('train.csv', index=False)
# valid.to_csv('valid.csv', index=False)

In [7]:
train.head()

,label,text
1001,Negative,menteri kabinet beliau sendiri bercakap mengen...
2494,Positive,"Selain itu, Muslimat Pas juga ingin mencadangk..."
1284,Negative,Apa yang boleh disimpulkan sekarang adalah ia ...
1151,Positive,tidak mahu ketinggalan daripada arus pembangun...
3630,Positive,Kita akan berhubung dengan pihak berkuasa Sing...


In [8]:
valid.head()

,label,text
2023,Positive,asean yang dianggotai sepuluh negara terletak ...
2587,Positive,"Pertubuhan Makanan dan Pertanian, sebuah agens..."
3222,Negative,usaha mengaburi mata dan perasaan orang Melayu
1263,Negative,jika terdapat unsur kecuaian hasil daripada pe...
781,Positive,Peruntukan bagi pembinaan bangunan sekolah bah...


In [9]:
def generate_bigrams(x):
#     x = x.split(' ')
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

In [8]:
generate_bigrams(train.iloc[0]['text'].split(' '))

['menteri',
 'kabinet',
 'beliau',
 'sendiri',
 'bercakap',
 'mengenai',
 'pembinaan',
 'hospital',
 'kerajaan',
 'sakit',
 'bekalan',
 'ubat',
 'di',
 'hospital',
 'kerajaan',
 'yang',
 'sering',
 'terputus',
 'dan',
 'pesakit',
 'yang',
 'dulu',
 'mampu',
 'berubat',
 'di',
 'klinik',
 'dan',
 'hospital',
 'swasta',
 'kini',
 'berpusu-pusu',
 'ke',
 'klinik',
 'dan',
 'hospital',
 'awam',
 'berubat di',
 'terputus dan',
 'di hospital',
 'kini berpusu-pusu',
 'yang dulu',
 'hospital awam',
 'pesakit yang',
 'pembinaan hospital',
 'dan hospital',
 'swasta kini',
 'yang sering',
 'menteri kabinet',
 'beliau sendiri',
 'mengenai pembinaan',
 'di klinik',
 'sakit bekalan',
 'bekalan ubat',
 'dan pesakit',
 'dulu mampu',
 'berpusu-pusu ke',
 'mampu berubat',
 'hospital swasta',
 'kerajaan yang',
 'kabinet beliau',
 'bercakap mengenai',
 'ubat di',
 'klinik dan',
 'kerajaan sakit',
 'ke klinik',
 'hospital kerajaan',
 'sendiri bercakap',
 'sering terputus']

In [10]:
import torch
from torchtext import data, datasets


SEED = 1234

torch.manual_seed(SEED)
# torch.cuda.manual_seed(SEED)
# torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy', preprocessing=generate_bigrams)
LABEL = data.LabelField(dtype=torch.float)


In [12]:
train_val_fields = [
    ('label', LABEL), # process it as label
    ('text', TEXT) # process it as text
]

trainds, valds = data.TabularDataset.splits(path='./data/', 
                                            format='csv', 
                                            train='train.csv', 
                                            validation='valid.csv', 
                                            fields=train_val_fields, 
                                            skip_header=True)

In [13]:
trainds.fields.items()

dict_items([('label', <torchtext.data.field.LabelField object at 0x1a18e55b00>), ('text', <torchtext.data.field.Field object at 0x1a18df30f0>)])

In [16]:
ex = trainds[0]

In [17]:
ex.text

['menteri',
 'kabinet',
 'beliau',
 'sendiri',
 'bercakap',
 'mengenai',
 'pembinaan',
 'hospital',
 'kerajaan',
 'sakit',
 'bekalan',
 'ubat',
 'di',
 'hospital',
 'kerajaan',
 'yang',
 'sering',
 'terputus',
 'dan',
 'pesakit',
 'yang',
 'dulu',
 'mampu',
 'berubat',
 'di',
 'klinik',
 'dan',
 'hospital',
 'swasta',
 'kini',
 'berpusu',
 '-',
 'pusu',
 'ke',
 'klinik',
 'dan',
 'hospital',
 'awam',
 'kabinet beliau',
 'di hospital',
 'terputus dan',
 'yang dulu',
 'mampu berubat',
 'kini berpusu',
 'sendiri bercakap',
 'pembinaan hospital',
 'berubat di',
 'ke klinik',
 'berpusu -',
 'ubat di',
 'yang sering',
 'pusu ke',
 'sering terputus',
 'hospital swasta',
 'di klinik',
 '- pusu',
 'hospital kerajaan',
 'hospital awam',
 'dan hospital',
 'klinik dan',
 'bercakap mengenai',
 'pesakit yang',
 'mengenai pembinaan',
 'kerajaan sakit',
 'beliau sendiri',
 'bekalan ubat',
 'kerajaan yang',
 'menteri kabinet',
 'dulu mampu',
 'swasta kini',
 'dan pesakit',
 'sakit bekalan']

In [58]:
ex.label

'Negative'

In [59]:
TEXT.build_vocab(trainds)
LABEL.build_vocab(trainds)

In [61]:
LABEL.vocab.itos

['Positive', 'Negative']

In [111]:
traindl, valdl = data.BucketIterator.splits(datasets=(trainds, valds), # specify train and validation Tabulardataset
                                            batch_sizes=(3,3),  # batch size of train and validation
#                                             sort_key=lambda x: len(x.SentimentText), # on what attribute the text should be sorted
                                            device=None, # -1 mean cpu and 0 or None mean gpu
#                                             sort_within_batch=True, 
                                            sort=False,
                                            repeat=False)

In [72]:
print(len(traindl), len(valdl))

983 246


In [77]:
batch = next(iter(traindl)) # BucketIterator return a batch object
# print(type(batch))
# torchtext.data.batch.Batch

print(batch.label) # labels of the batch
# tensor([ 0,  0,  0], device='cuda:0')

print(batch.text) # text index and length of the batch
# (tensor([[  204,   107,   956],
#         [   29,  3176,   112],
#         [ 4391,   195,    28],
#         [ 1413,    57,    57],
#         [   19,    32,    26],
#         [   32,   114,  5138]], device='cuda:0'),
# tensor([ 6,  6,  6], device='cuda:0'))

# print(batch.dataset.fields)

tensor([1., 1., 1.])
tensor([[17523,  3682,    35],
        [  841,   111,     7],
        [23110,     9,  4615],
        [  285,   111,  1278],
        [ 3422,   688,    17],
        [ 4952,     6,     7],
        [26033,    91,  1960],
        [ 1422,    12,  1276],
        [  200, 28791,   433],
        [   31,    24,   146],
        [ 2546,    19,  1367],
        [19364,  1073,     3],
        [  715,   891,  1006],
        [  132,     3,   701],
        [    9,   172,   722],
        [  132, 30344, 31767],
        [  112,    24, 41882],
        [  173,  1173, 27476],
        [    7,  1622, 34515],
        [ 2953,  4701, 17363],
        [ 2573,  1906,  1322],
        [ 2566,    68, 34278],
        [  200,   275,  9619],
        [  760,   234, 33189],
        [  132, 17213,  2200],
        [  450,     5,     1],
        [    7, 28476,     1],
        [  537, 40373,     1],
        [  742, 17214,     1],
        [   30,  6324,     1],
        [ 1954, 30345,     1],
        [ 8657, 26

In [78]:

import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        embedded = self.embedding(x)
                
        #embedded = [sent len, batch size, emb dim]
        
        embedded = embedded.permute(1, 0, 2)
        
        #embedded = [batch size, sent len, emb dim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        
        #pooled = [batch size, embedding_dim]
                
        return self.fc(pooled)

In [114]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM)

In [115]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [116]:
criterion = nn.BCEWithLogitsLoss()

In [117]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [118]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [119]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [120]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, traindl, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valdl, criterion)
#     print(train_loss, train_acc)
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')



| Epoch: 01 | Train Loss: 0.654 | Train Acc: 64.43% | Val. Loss: 0.619 | Val. Acc: 66.40% |
| Epoch: 02 | Train Loss: 0.577 | Train Acc: 66.77% | Val. Loss: 0.574 | Val. Acc: 68.02% |
| Epoch: 03 | Train Loss: 0.441 | Train Acc: 80.84% | Val. Loss: 0.535 | Val. Acc: 73.17% |
| Epoch: 04 | Train Loss: 0.289 | Train Acc: 92.51% | Val. Loss: 0.510 | Val. Acc: 74.80% |
| Epoch: 05 | Train Loss: 0.176 | Train Acc: 97.52% | Val. Loss: 0.501 | Val. Acc: 74.12% |


In [112]:
valid_loss, valid_acc = evaluate(model, valdl, criterion)

In [ ]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')

In [92]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(sentence):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:

predict_sentiment("This film is terrible")

In [95]:
valid.iloc[0]['text'], valid.iloc[0]['label']

('asean yang dianggotai sepuluh negara terletak di rantau asia tenggara yang mengandungi kepelbagaian agama bangsa dan budaya serta banyak jurang perbezaan dalam kalangan anggota asean dari aspek ekonomi politik keselamatan dan sosio-budaya',
 'Positive')

In [96]:
valid.iloc[1]['text'], valid.iloc[1]['label']

('Pertubuhan Makanan dan Pertanian, sebuah agensi Pertubuhan Bangsa-Bangsa Bersatu, menjangka pada tahun 2050 penduduk dunia akan mencecah 9.1 bilion orang dan untuk menepati permintaan, pengeluaran makanan perlu ditingkatkan sebanyak 70 peratus.',
 'Positive')

In [97]:
valid.iloc[2]['text'], valid.iloc[2]['label']

('usaha mengaburi mata dan perasaan orang Melayu', 'Negative')

In [99]:
valid.iloc[3]['text'], valid.iloc[3]['label']

('jika terdapat unsur kecuaian hasil daripada penyiasatan itu, maka, kakitangan yang terlibat akan dikenakan tindakan tegas.',
 'Negative')

In [101]:
valid.iloc[4]['text'], valid.iloc[4]['label']

('Peruntukan bagi pembinaan bangunan sekolah baharu itu nanti akan dimasukkan bersama dalam projek memperbaiki sekolah yang terjejas teruk akibat banjir, baru-baru ini',
 'Positive')

In [105]:
valid.iloc[5]['text'], valid.iloc[5]['label']

('sebaliknya kebajikan mereka jadi bertambah teruk akibat penghapusan subsidi dan kejatuhan kadar tukaran ringgit yang secara langsung menyebabkan harga barang dan perkhidmatan naik',
 'Positive')

In [106]:
predict_sentiment(valid.iloc[5]['text'])

0.9949637651443481

In [122]:
LABEL.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index()>,
            {'Positive': 0, 'Negative': 1})